In [1]:
%%time
#!/usr/bin/env python

# Thanks to Peter Schmidtke for the guts of this script
# https://pschmidtke.github.io/blog/rdkit/crystallography/small%20molecule%20xray/xray/database/2021/01/25/cod-and-torsion-angles.html

import pandas as pd
import numpy as np
import glob

from rdkit import Chem
from rdkit.Chem import rdMolTransforms

# ignore warnings
from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*')

# from ETKDG paper: https://pubs.acs.org/doi/abs/10.1021/acs.jcim.5b00654
torsions=pd.read_table("list_torsion_patterns.txt",header=None,usecols=[1])
# from Ring ETKDG paper: https://pubs.acs.org/doi/10.1021/acs.jcim.0c00025
# torsions=pd.read_table("ring_smarts_patterns.txt",header=None,usecols=[1])

# filename template for output, e.g. t1.txt, t2.txt, etc.
out_template = 't{}.txt'

# patterns=torsions[1]
# for debugging
patterns=torsions[1][:-1]

# bin size (in degrees)
bin_size = 1
bins = round(360 / bin_size)

# This outer loop is for each one of the torsion patterns
# We compile the pattern, then loop through the SDF / XYZ files
index = 0
for torsionSmarts in patterns:
    index += 1
    print(index, torsionSmarts)

    angles = np.zeros(bins) # create a histogram of angles with X degree bins
    total_matches = 0

    torsionQuery = Chem.MolFromSmarts(torsionSmarts)

    # these SMARTS have atom maps, so convert them
    # http://www.rdkit.org/docs/GettingStartedInPython.html#atom-map-indices-in-smarts
    index_map = {}
    for atom in torsionQuery.GetAtoms() :
        map_num = atom.GetAtomMapNum()
        if map_num:
            index_map[map_num-1] = atom.GetIdx()

    map_list = [index_map[x] for x in sorted(index_map)]
    
    try:
        # loop through the files and then each of the molecules in the file (if more than one)
        for sd_file in glob.iglob("../pqr/*/*-N.sdf"):
            suppl = Chem.SDMolSupplier(sd_file, removeHs=False)

            for mol in suppl:

                if mol is None: continue

                # get the 3D geometry
                conf = mol.GetConformer(0)

                matches = mol.GetSubstructMatches(torsionQuery)
                for match in matches:
                    total_matches += 1 # to normalize

                    # get the atom maps from the SMARTS match
                    mapped = [match[x] for x in map_list]
                    angle = rdMolTransforms.GetDihedralDeg(conf, mapped[0],mapped[1],mapped[2],mapped[3])
                    if (angle < 0.0):
                        angle += 360.0

                    # okay, we want to hash - e.g., 5° bins    
                    angle = round(angle / bin_size) % bins

                    angles[angle] += 1

        if total_matches > 0:
            # angles = angles / total_matches # normalize
            np.savetxt(out_template.format(index), angles, fmt='%.3e', delimiter=',')
            # print(angles)
    except:
        print('Issue with {} - {}'.format(index, torsionSmarts))

1 [O:1]=[C:2]!@;-[O:3]~[CH0:4]
2 [O:1]=[C:2]([N])!@;-[O:3]~[C:4]
3 [O:1]=[C:2]!@;-[O:3]~[C:4]
4 [O:1]=[C:2]!@;-[O:3]~[!#1:4]
5 O=[C:1][O:2]!@;-[c:3]~[*:4]
6 O=[C:1][O:2]!@;-[CX3:3]~[*:4]
7 O=[C:1][O:2]!@;-[CH1:3][H:4]
8 O=[C:1][O:2]!@;-[CH2:3]~[C:4]
9 [H:1][CX4H1:2]!@;-[O:3][CX4:4]
10 [C:1][CH2:2]!@;-[O:3][CX4:4]
11 [*:1][CX4:2]!@;-[O:3][CX3:4](=[!O])
12 [O:1][CX4:2]!@;-[O:3][CX4:4]
13 [*:1][CX4:2]!@;-[O:3][CX4:4]
14 [cH1:1][c:2]([cH1])!@;-[O:3][S:4]
15 [cH1:1][c:2]([cH0])!@;-[O:3][S:4]
16 [cH0:1][c:2]([cH0])!@;-[O:3][S:4]
17 [cH1:1][c:2]([cH1])!@;-[O:3][c:4]
18 [cH1:1][c:2]([cH0])!@;-[O:3][c:4]
19 [cH0:1][c:2]([cH0])!@;-[O:3][c:4]
20 [cH0:1][c:2]([cH0])!@;-[O:3][!C;!H:4]
21 [cH0:1][c:2]([cH1])!@;-[O:3][!C;!H:4]
22 [cH1:1][c:2]([cH1])!@;-[O:3][!C;!H:4]
23 [cH:1][c:2]([cH])!@;-[O:3][C:4](F)(F)[F]
24 [cH0:1][c:2]([cH0])!@;-[O:3][CX4:4]([!#1])([!#1])([!#1])
25 [a:1][c:2]([a])!@;-[O:3][CX4H0:4]
26 [cH1,n:1][c:2]!@;-[O:3][CRH1:4]
27 [cH1,n:1][c:2]!@;-[O:3][CH1:4]
28 [nX2H0:1][c:2]([cH0])!@;

202 [*:1][S:2](=O)(=O)!@;-[NX3H0:3][*:4]
203 [!#1:1][CX3:2]!@;-[SX2:3][!#1:4]
204 [!#1:1][CX4:2]!@;-[SX2:3][!#1:4]
205 [!#1:1][CX3:2]!@;-[SX3:3][!#1:4]
206 [!#1:1][CX4:2]!@;-[SX3:3][!#1:4]
207 [!#1:1][CX3:2]!@;-[SX4:3][!#1:4]
208 [H:1][CX4H1:2]!@;-[SX4:3][!#1:4]
209 [!#1:1][CX4:2]!@;-[SX4:3][!#1:4]
210 [aH1:1][c:2]([aH1])!@;-[SX2:3][!#1:4]
211 [aH1:1][c:2]([aH0])!@;-[SX2:3][!#1:4]
212 [aH0:1][c:2]([aH0])!@;-[SX2:3][!#1:4]
213 [!#1:1][c:2]!@;-[SX2:3][!#1:4]
214 [!#1:1][c:2]!@;-[SX3:3][!#1:4]
215 [aH1:1][c:2]([aH1])!@;-[SX4:3][!#1:4]
216 [aH0:1][c:2]([aH1])!@;-[SX4:3][!#1:4]
217 [aH0:1][c:2]([aH0])!@;-[SX4:3][!#1:4]
218 [!#1:1][c:2]!@;-[SX4:3][!#1:4]
219 [O:1]=[CX3:2]([NH1])!@;-[CH2:3][CX3:4]=O
220 [O:1]=[CX3:2]([NH1])!@;-[CH2:3][C:4]
221 [C]\[CX3:1]([H])=[CX3:2]([C])!@;-\[CH2:3][C:4]
222 [C]\[CX3:1]([H])=[CX3:2]([H])!@;-\[CH1:3](C)[C:4]
223 [C]\[CX3:1]([H])=[CX3:2]([H])!@;-\[CH2:3][C:4]
224 [C]\[CX3:1]([H])=[CX3:2]([H])!@;-/[CH2:3][C:4]
225 [N:1][C:2](=O)!@;-[CX4H2:3][CX4H2:4]
226 N[C:2

In [9]:
%%time
#!/usr/bin/env python

# Thanks to Peter Schmidtke for the guts of this script
# https://pschmidtke.github.io/blog/rdkit/crystallography/small%20molecule%20xray/xray/database/2021/01/25/cod-and-torsion-angles.html

import pandas as pd
import numpy as np
import glob

from rdkit import Chem
from rdkit.Chem import rdMolTransforms

# ignore warnings
from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*')

# from ETKDG paper: https://pubs.acs.org/doi/abs/10.1021/acs.jcim.5b00654
torsions=pd.read_table("list_torsion_patterns.txt",header=None,usecols=[1])
# from Ring ETKDG paper: https://pubs.acs.org/doi/10.1021/acs.jcim.0c00025
# torsions=pd.read_table("ring_smarts_patterns.txt",header=None,usecols=[1])

# filename template for output, e.g. t1.txt, t2.txt, etc.
out_template = 't{}.txt'

# patterns=torsions[1]
# for debugging
patterns=torsions[1][:-1]
# index = torsions[1]
# print(patterns)
print(patterns[19])
# # bin size (in degrees)
# bin_size = 1
# bins = round(360 / bin_size)

# # This outer loop is for each one of the torsion patterns
# # We compile the pattern, then loop through the SDF / XYZ files
# index = 0
# for torsionSmarts in patterns:
#     index += 1
#     print(index, torsionSmarts)

#     angles = np.zeros(bins) # create a histogram of angles with X degree bins
#     total_matches = 0

#     torsionQuery = Chem.MolFromSmarts(torsionSmarts)

#     # these SMARTS have atom maps, so convert them
#     # http://www.rdkit.org/docs/GettingStartedInPython.html#atom-map-indices-in-smarts
#     index_map = {}
#     for atom in torsionQuery.GetAtoms() :
#         map_num = atom.GetAtomMapNum()
#         if map_num:
#             index_map[map_num-1] = atom.GetIdx()

#     map_list = [index_map[x] for x in sorted(index_map)]
    
#     try:
#         # loop through the files and then each of the molecules in the file (if more than one)
#         for sd_file in glob.iglob("../pqr/*/*-N.sdf"):
#             suppl = Chem.SDMolSupplier(sd_file, removeHs=False)

#             for mol in suppl:

#                 if mol is None: continue

#                 # get the 3D geometry
#                 conf = mol.GetConformer(0)

#                 matches = mol.GetSubstructMatches(torsionQuery)
#                 for match in matches:
#                     total_matches += 1 # to normalize

#                     # get the atom maps from the SMARTS match
#                     mapped = [match[x] for x in map_list]
#                     angle = rdMolTransforms.GetDihedralDeg(conf, mapped[0],mapped[1],mapped[2],mapped[3])
#                     if (angle < 0.0):
#                         angle += 360.0

#                     # okay, we want to hash - e.g., 5° bins    
#                     angle = round(angle / bin_size) % bins

#                     angles[angle] += 1

#         if total_matches > 0:
#             # angles = angles / total_matches # normalize
#             np.savetxt(out_template.format(index), angles, fmt='%.3e', delimiter=',')
#             # print(angles)
#     except:
#         print('Issue with {} - {}'.format(index, torsionSmarts))

[cH0:1][c:2]([cH0])!@;-[O:3][!C;!H:4]
CPU times: user 2.5 ms, sys: 0 ns, total: 2.5 ms
Wall time: 2.07 ms
